In [1]:
# 🔍 DIAGNOSTIC: Investigate Parameter Count Anomaly
import numpy as np
import pickle
import json
import os

print("=" * 70)
print("🔍 DIAGNOSTIC: Parameter Count & Architecture Investigation")
print("=" * 70)

# 1. Load all training histories and inspect
history_files = {
    'baseline': '../results/training_history.json',
    'enhanced': '../results/enhanced_training_history.json',
    'custom':   '../results/custom_training_history.json'
}

histories = {}
for name, path in history_files.items():
    if os.path.exists(path):
        with open(path, 'r') as f:
            histories[name] = json.load(f)
        h = histories[name]
        print(f"\n{'='*50}")
        print(f"📋 {name.upper()} MODEL HISTORY:")
        print(f"   Parameters:      {h.get('total_parameters', 'N/A'):,}" 
              if isinstance(h.get('total_parameters'), int) 
              else f"   Parameters:      {h.get('total_parameters', 'N/A')}")
        print(f"   Architecture:    {h.get('architecture', 'N/A')}")
        print(f"   Model type:      {h.get('model_type', h.get('baseline_model_type', 'N/A'))}")
        print(f"   Data Aug:        {h.get('data_augmentation', 'N/A')}")
        print(f"   LR Scheduling:   {h.get('learning_rate_scheduling', 'N/A')}")
        print(f"   Total epochs:    {h.get('total_epochs', 'N/A')}")
        print(f"   Best val acc:    {h.get('best_val_accuracy', 'N/A'):.4f}")
        print(f"   Test accuracy:   {h.get('final_test_accuracy', 'N/A'):.4f}")
        print(f"   Best epoch:      {h.get('best_epoch', 'N/A')}")
    else:
        print(f"\n❌ {name}: history file NOT FOUND at {path}")

# 2. Check which model files actually exist
print(f"\n{'='*50}")
print("📁 MODEL FILES ON DISK:")
model_dir = '../models/'
if os.path.exists(model_dir):
    for f in sorted(os.listdir(model_dir)):
        size_mb = os.path.getsize(os.path.join(model_dir, f)) / 1e6
        print(f"   {f:45s} ({size_mb:.1f} MB)")
else:
    print("   ❌ ../models/ directory not found!")

# 3. Check results directory
print(f"\n📁 RESULTS FILES ON DISK:")
results_dir = '../results/'
if os.path.exists(results_dir):
    for f in sorted(os.listdir(results_dir)):
        size_kb = os.path.getsize(os.path.join(results_dir, f)) / 1e3
        print(f"   {f:45s} ({size_kb:.1f} KB)")

# 4. Key anomaly check
print(f"\n{'='*50}")
print("🚨 ANOMALY DETECTION:")

if 'baseline' in histories and 'enhanced' in histories:
    b_params = histories['baseline'].get('total_parameters', 0)
    e_params = histories['enhanced'].get('total_parameters', 0)
    c_params = histories.get('custom', {}).get('total_parameters', 0)
    
    print(f"   Baseline parameters:  {b_params:>12,}")
    print(f"   Enhanced parameters:  {e_params:>12,}")
    print(f"   Custom parameters:    {c_params:>12,}")
    
    if b_params == e_params:
        print("\n   ⚠️  CONFIRMED: Baseline and Enhanced have IDENTICAL parameter counts!")
        print("   This means they are likely the SAME architecture.")
        print("   The Baseline was probably built with the Enhanced architecture.")
    elif abs(b_params - e_params) / max(b_params, e_params) < 0.05:
        print("\n   ⚠️  WARNING: Baseline and Enhanced parameters differ by <5%")
        print("   They may be functionally identical architectures.")
    else:
        print("\n   ✅ Parameter counts differ as expected.")
    
    # Expected values
    print(f"\n   Expected Baseline:  ~1,400,000 (3-layer CNN + Flatten)")
    print(f"   Expected Enhanced:  ~2,800,000 (8-layer CNN + GAP)")
    print(f"   Expected Custom:    ~2,800,000 (8-layer CNN + GAP)")
    
    # Diagnose what actually happened
    if 1_300_000 <= c_params <= 1_600_000:
        print(f"\n   💡 Custom (~{c_params:,}) matches Baseline spec (~1.4M)")
    if 2_500_000 <= b_params <= 3_200_000:
        print(f"   💡 Baseline (~{b_params:,}) matches Enhanced spec (~2.8M)")
        print("   → The Baseline was accidentally built with the Enhanced architecture!")
    if 2_500_000 <= e_params <= 3_200_000:
        print(f"   💡 Enhanced (~{e_params:,}) matches Enhanced spec (~2.8M) ✅")

🔍 DIAGNOSTIC: Parameter Count & Architecture Investigation

📋 BASELINE MODEL HISTORY:
   Parameters:      7,928,987
   Architecture:    3_layer_cnn_with_flatten
   Model type:      baseline_cnn
   Data Aug:        False
   LR Scheduling:   False
   Total epochs:    38
   Best val acc:    0.3808
   Test accuracy:   0.3833
   Best epoch:      27

📋 ENHANCED MODEL HISTORY:
   Parameters:      1,448,443
   Architecture:    double_conv_blocks_with_gap
   Model type:      enhanced_cnn
   Data Aug:        True
   LR Scheduling:   True
   Total epochs:    53
   Best val acc:    0.4644
   Test accuracy:   0.4500
   Best epoch:      37

📋 CUSTOM MODEL HISTORY:
   Parameters:      1,448,443
   Architecture:    custom_spectrogram_generator
   Model type:      custom_cnn
   Data Aug:        True
   LR Scheduling:   True
   Total epochs:    30
   Best val acc:    0.2902
   Test accuracy:   0.2500
   Best epoch:      14

📁 MODEL FILES ON DISK:
   best_baseline_model.keras                     (95.2 MB